In [1]:
import shutil
import datetime
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')

from chart import *
from table import *
from docx.text.run import *

In [2]:
# # DRM 방지(default.docx)
# def findfile(name, path):
#     for dirpath, dirname, filename in os.walk(path):
#         if name in filename:
#             return os.path.join(dirpath, name)

# file_path=os.path.abspath("")
# srcpath = findfile("default.docx.org", file_path)
# dir, file = os.path.split(srcpath)
# shutil.copy2(srcpath, dir+"\default.docx")

In [3]:
from document import *
# report version
rtype = '월간' # 연간, 반기

In [4]:
region_list = [ 'KR', 'CN', 'NA', 'EU', 'SG', 'RU' ]
region = 'All'
#region = 'KR' # All|KR|CN|NA|EU|SG|RU
#region = 'CN'
#region = 'NA'
#region = 'EU'
#region = 'SG'
#region = 'RU'
# RU쪽은 SG로 나오는 버그 존재

In [5]:
month = '7월'

In [6]:
# 메타 정보

today = datetime.date.today()

y = today.year
m = today.month
d = today.day
w = today.weekday()

v = '1'

if region == 'All':
    regex = '(KR|CN|NA|EU|SG|RU|Global|All)'
else:
    regex = region

In [7]:
# Document 기본 폰트
style = document.styles['Normal']
style.font.name = '맑은고딕'
style.font.size = Pt(12)
style._element.rPr.rFonts.set(qn('w:eastAsia'), '맑은고딕')

In [8]:
# Chart 기본 폰트
alt.themes.register('맑은고딕', hanfont)
alt.themes.enable('맑은고딕')

ThemeRegistry.enable('맑은고딕')

## 주요 이벤트

In [9]:
# ETL
base_path = r"./data/요약_202207_서비스관리_v1.ods"
#base_path = r"./data/주요이벤트_202206_서비스관리_0811_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)

# 리전 & 운영계, 월간보고서용
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed_rows = len(df_completed)
completed = counter(df_completed, regex)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted_rows = len(df_incompleted)
incompleted = counter(df_incompleted, regex)

In [10]:
# Chart
if region == 'All':
    # Data
    pivot, total = getPivotTable_event(df_filtered, month, region_list)
    source = pivot
    
    # Chart
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart(source)
    chart.save(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')
    document.add_paragraph('주요 이벤트')
    # Document
    document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 미완료: {incompleted})', style='List Bullet')
    document.add_picture(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')
else:
    document.add_paragraph('주요 이벤트')
    document.add_paragraph(f'완료: {completed} 건 / 이슈: {incompleted} 건', style='List Bullet')
    pass

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [11]:
pivot

,리전,합계,완료,미완료,비중
0,KR,0,0,0,NaN
1,CN,0,0,0,NaN
2,NA,0,0,0,NaN
3,EU,0,0,0,NaN
4,SG,0,0,0,NaN
5,RU,0,0,0,NaN


In [ ]:
pivot

In [15]:
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

In [16]:
document.save('demo.docx')

## 1.1 모니터링(인시던트핸들링)

In [17]:
base_path = r"./data/1_1_202207_모니터링_인시던트핸들링_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df['장애전파소요시간'] = (pd.to_datetime(df['장애전파시간.1'])-pd.to_datetime(df['발생인지시간'])).astype('timedelta64[m]')

# Pivoting
pivot, total = getPivotTable(df, month)

# 리전 & 운영계, 월간보고서용
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]
df_filtered, timeslot = getTables(df_filtered, regex, month)
    
# SLA failed
df_failed = df_filtered[ (df_filtered['장애전파'] > 15) & (df_filtered['월']== month)]
df_failed[['날짜', '장애이벤트', '조치내용', 'JIRA_Ticket_No', '장애전파소요시간']]

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [15]:
df_filtered

,년도,월,리전,AZ,테넌트,진행상태,성패,작업내용,주간,월간
0,22년,1월,SG,1센터,PRD,완료,성공,(SG 리전) H53 모니터링 이관 (2021.10.03)\n==> 메뉴일 갱신 본...,O,O
2,22년,1월,SG,1센터,PRD,완료,성공,(SG) DB 이중화 미구성\n이중화 구성 위한 Heartbeat 전용 네트워크 구...,O,O
4,22년,2월,EU,1센터,PRD,완료,성공,(EU) 방화벽 Capa 부족으로 인한 방화벽 장비 교체\n(방화벽) 출퇴근 시간 ...,O,O
9,22년,3월,EU,1센터,PRD,완료,성공,(EU) 방화벽 이슈로 예외 처리된(제외되었던) 기능 원상 복구 (적용)\n(완료)...,O,O
10,22년,4월,Global,1센터,PRD,완료,성공,(모니터링) Zabbix 모니터링 항목 조정\n(이슈) 중요도 무관하게 너무 많은 ...,O,O
13,22년,4월,NA,1센터,PRD,완료,성공,(NA) Trident 표준 프로세스 정립 및 운영 이관 \n북미 운영계 서비스허브...,O,O
14,22년,4월,Global,1센터,PRD,완료,성공,(공통) 운영 DB 실태 파악 ::: (COC 설치/구성 아닌) 운영계 DB 서버 ...,O,O
15,22년,4월,KR,2센터,PRD,완료,성공,(내수) 상암 센터 운영 업무 이관\n업무 인수인계 체크리스트\n(완료) 데이터센터...,O,O
16,22년,5월,KR,1센터,PRD,완료,성공,(KR) LB [A10(5840) → F5]이관\n'22년 운영계 A10(5840)...,O,O
17,22년,5월,KR,2센터,PRD,완료,성공,(KR) LB [A10(5840) → F5]이관\n'22년 운영계 A10(5840)...,O,O


In [18]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart1(source)
    chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 스캐터 차트    
chart = getScatterChart(timeslot, month)
chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_2.png')

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_3.png')

In [19]:
# Document
document.add_paragraph('모니터링(인시던트핸들링)', style='List Number')
document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 미완료: {incompleted})', style='List Bullet')
if region == 'All':
    document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_3.png')
document.add_page_break()

# Document
document.add_paragraph(f'일별 장애전파 소요시간(분) 현황', style='List Bullet')
document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_2.png')

##  (향후) 평균 소요 시간, 최대 소요 시간 추가 하자!!!
document.add_paragraph(f'장애전파SLA(15분) 초과: {len(df_failed)} 건', style='List Bullet')
addFailedTable(df_failed, regex, month)

p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)


## 1.2 이슈 관리

In [20]:
base_path = r"./data/1_2_202207_이슈관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & ((df['진행상태']=='완료')|(df['진행상태']=='진행중')) & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [21]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart1(source)
    chart.save(f'./charts/im_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/im_{region}_{y}_{m}_{d}_2.png')

In [22]:
# Document
total_sum = completed + incompleted
document.add_paragraph('이슈 관리', style='List Number')
document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 진행: {incompleted})', style='List Bullet')

if region == 'All':
    document.add_picture(f'./charts/im_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Document
addTable4(df_filtered, regex, month)

p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/im_{region}_{y}_{m}_{d}_2.png')
document.add_page_break()

## 1.3 장애 관리

In [23]:
base_path = r"./data/1_3_202207_장애관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [24]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart1(source)
    chart.save(f'./charts/pm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/pm_{region}_{y}_{m}_{d}_2.png')

In [25]:
# Document
document.add_paragraph('장애(RCA) 관리', style='List Number')
document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 진행: {incompleted})', style='List Bullet')

if region == 'All':
    document.add_picture(f'./charts/pm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Document
addTable4(df_filtered, regex, month)

p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/pm_{region}_{y}_{m}_{d}_2.png')
document.add_page_break()

## 1.4 변경 관리

In [26]:
base_path = r"./data/1_4_202207_변경관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [27]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart1(source)
    chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')

In [28]:
df_filtered = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
total = len(df_filtered)

In [29]:
# Document
document.add_paragraph('변경 관리', style='List Number')
document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 진행: {incompleted})', style='List Bullet')
    
if region == 'All':
    document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Document
p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

document.add_paragraph('월별 추세', style='List Bullet')
document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')

document.add_paragraph('상세 현황', style='List Bullet')
addTable3(df_filtered, regex, month)
document.add_page_break()

## 1.5 요청 관리

In [30]:
base_path = r"./data/1_5_202207_요청관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [31]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart1(source)
    chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')

In [32]:
# Document
document.add_paragraph('요청 관리', style='List Number')
document.add_paragraph(f'총 {completed+incompleted}건 (완료)', style='List Bullet')

if region == 'All':
    document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Document
# addTable3(df_filtered, regex, month)
document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')
document.add_page_break()

## 1.6 자산 관리 - VM

In [33]:
base_path = r"./data/1_6_202207_자산관리_VM_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_ = df_filtered[ (df_filtered['월']==month) ]
#total = len(df_)
total = df_['count'].sum()

In [34]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart1(source)
    chart.save(f'./charts/vm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2s(df_filtered, month)
chart.save(f'./charts/vm_{region}_{y}_{m}_{d}_2.png')

WARN text dropped as it is incompatible with "point".


In [35]:
chart

alt.LayerChart(...)

In [36]:
# Document
document.add_paragraph('자산 관리(vm)', style='List Number')
document.add_paragraph(f'총 {int(total)}인스턴스 (운영계)', style='List Bullet')

if region == 'All':
    document.add_picture(f'./charts/vm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Document
# addTable(df_filtered, regex, month)
document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/vm_{region}_{y}_{m}_{d}_2.png')
document.add_page_break()

## 1.6 자산 관리 - DB

In [37]:
base_path = r"./data/1_6_202207_자산관리_DB_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df_filtered, month)
pivot2, total = getPivotTable2(df_filtered, month)
pivot2 = pivot2.reset_index()

df_ = df_filtered[ (df_filtered['월']==month) ]
#total = len(df_)
total = df_['count'].sum()

In [38]:
pivot

,리전,count,합계,비중
2,KR,474.0,474.0,70.4
5,SG,57.0,57.0,8.5
1,EU,48.0,48.0,7.1
0,CN,43.0,43.0,6.4
3,NA,43.0,43.0,6.4
4,RU,8.0,8.0,1.2


In [39]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart1(source)
    chart.save(f'./charts/db_{region}_{y}_{m}_{d}_1.png')
else:
    pass

source = df_filtered
# chart = getBarChart2s(source, month)
#chart = getStackedHBarChart1(source)
chart = getStackedVBarChart1(source)
chart.save(f'./charts/db_{region}_{y}_{m}_{d}_2.png')

# Chart - 라인 차트
chart = getLineChart2s(source, month)
chart.save(f'./charts/db_{region}_{y}_{m}_{d}_3.png')

WARN text dropped as it is incompatible with "point".


In [40]:
df_filtered.head()

,년도,월,리전,AZ,테넌트,유형,count,주간,월간,연간
0,22년,1월,KR,1센터,PRD,PostgreSQL,110.0,O,O,O
1,22년,1월,KR,1센터,PRD,Tibero,28.0,O,O,O
2,22년,1월,KR,1센터,PRD,Maria,32.0,O,O,O
3,22년,1월,KR,1센터,PRD,Redis,97.0,O,O,O
4,22년,1월,KR,1센터,PRD,Mongo,84.0,O,O,O


In [41]:
# source = df_filtered
# bar = alt.Chart(source).mark_bar().encode(
#     x=alt.X('월:N'),
#     y=alt.Y('sum(count)', title=''),
#     color=alt.Color("유형:N", legend=None)
# )
# line = alt.Chart(source).mark_line(color="red").encode(
#     x=alt.X('월', title='월별 추세'),
#     y=alt.Y('sum(count)', title='')
# )
# text = bar.mark_text(
#     size=12,
#     align='center',
#     baseline='line-top',
#     color='black',
#     dy=-20,
#     fontSize=12,
# ).encode(
#     text = 'sum(count):Q'
# )

In [42]:
document.add_paragraph('자산 관리(db)', style='List Number')
document.add_paragraph(f'총 {int(total)}인스턴스(운영계)', style='List Bullet')

if region == 'All':
    document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_1.png')
    
    # Document
    addPivot2Table(pivot2)
else:
    pass

document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_2.png')
p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

# document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_2.png')
document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_3.png')
document.add_page_break()

## 1.6 자산 관리 - K8s

In [43]:
base_path = r"./data/1_6_202207_자산관리_k8s_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_month = df_filtered[ (df_filtered['월']==month) ]
total = len(df_month)

In [44]:
# Chart - Bar 차트
if region == 'All':
    source = pivot
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart1(source)
    chart.save(f'./charts/k8s_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# source = df_
# chart = getBarChart(source)
# chart.save(f'./charts/k8s_{region}_{y}_{m}_{d}_2.png')
    
# Chart - Line 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/k8s_{region}_{y}_{m}_{d}_3.png')

In [45]:
chart

alt.LayerChart(...)

In [46]:
# Document
document.add_paragraph('자산 관리(k8s)', style='List Number')
document.add_paragraph(f'총 {total}노드(운영계)', style='List Bullet')

# Chart - Bar 차트
if region == 'All':
    document.add_picture(f'./charts/k8s_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Document
document.add_paragraph('상세 현황', style='List Bullet')
addTablek8s(df_filtered, regex, month)
#document.add_picture(f'./charts/k8s_{region}_{y}_{m}_{d}_2.png')

p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

document.add_paragraph('월별 추세', style='List Bullet')
document.add_picture(f'./charts/k8s_{region}_{y}_{m}_{d}_3.png')   
document.add_page_break()

## 1.7 용량 관리

In [47]:
base_path = r"./data/1_7_202207_용량관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable_capacity(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

incompleted = len (df_filtered[(df_filtered['진행상태']!='완료') & (df_filtered['월']==month)])

In [48]:
print(incompleted)

15


In [49]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart1(source)
    chart.save(f'./charts/cpm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/cpm_{region}_{y}_{m}_{d}_2.png')

In [50]:
# Document
document.add_paragraph('용량 관리', style='List Number')
document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 이슈: {incompleted})', style='List Bullet')

if region == 'All':
    document.add_picture(f'./charts/cpm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Document
document.add_paragraph(f'자원 이슈 현황', style='List Bullet')
addTable3(df_filtered, regex, month)

p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/cpm_{region}_{y}_{m}_{d}_2.png')   
document.add_page_break()

## 1.8 백업 관리

In [51]:
base_path = r"./data/1_8_202207_백업관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['성패']=='O') ]
completed_rows = len(df_completed)
completed = counter(df_completed, regex)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['성패']!='O') ]
incompleted_rows = len (df_incompleted)
incompleted = counter(df_incompleted, regex)

In [52]:
df.describe()

,연도,월,리전,AZ,테넌트,유형,성패,Instance,IP,백업 주기_full_archive,비고,주간,월간,연간
count,308,308,308,308,308,308,308,308,308,112,143,308,308,308
unique,1,1,6,2,1,5,2,302,298,4,20,1,1,1
top,2022년,7월,KR,1센터,PRD,PostgreSQL,O,aps-ssd-mongodb01.ap-northeast-2.prd.h53,172.17.36.205,주1회/매10분,standby,O,O,O
freq,308,308,222,287,308,131,241,4,2,76,54,308,308,308


In [53]:
df_filtered.describe()

,연도,월,리전,AZ,테넌트,유형,성패,Instance,IP,백업 주기_full_archive,비고,주간,월간,연간
count,308,308,308,308,308,308,308,308,308,112,143,308,308,308
unique,1,1,6,2,1,5,2,302,298,4,20,1,1,1
top,2022년,7월,KR,1센터,PRD,PostgreSQL,O,aps-ssd-mongodb01.ap-northeast-2.prd.h53,172.17.36.205,주1회/매10분,standby,O,O,O
freq,308,308,222,287,308,131,241,4,2,76,54,308,308,308


In [54]:
df_filtered_selectedColumns=df_filtered[['월', '리전', '유형', '성패', 'Instance', '비고']]
df_filtered_selectedColumns.sort_values(by=['월', '리전', '유형'], ascending=False, inplace=True)
df_filtered_selectedColumns.reset_index(drop=True, inplace=True)

addTable(df_filtered_selectedColumns)

In [55]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart1(source)
    chart.save(f'./charts/bm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart3(df_filtered, month)
#chart = getBarChart2(df_filtered, month)
chart.save(f'./charts/bm_{region}_{y}_{m}_{d}_2.png')

In [56]:
chart

alt.LayerChart(...)

In [57]:
# Document
document.add_paragraph('백업 관리', style='List Number')
document.add_paragraph(f'총: {completed+incompleted}건 (성공: {completed}/실패[미진행포함]: {incompleted})', style='List Bullet')

#if region == 'All':
#    document.add_picture(f'./charts/bm_{region}_{y}_{m}_{d}_1.png')
#else:
#    pass

# Document
document.add_picture(f'./charts/bm_{region}_{y}_{m}_{d}_2.png')
document.add_page_break()

## 1.9 보안 관리

In [58]:
base_path = r"./data/1_9_202207_보안관리_v1.1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable_security(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [59]:
df

,년도,월,리전,AZ,테넌트,진행상태,Ticket,제목,유형,주간,월간,연간
0,22년,1월,KR,1센터,DEV,완료,KRCLOUD-4134,EV 플랫폼 인프라 보안 점검 요청의 건,보안점검,O,O,O
1,22년,1월,KR,1센터,STG,완료,KRCLOUD-4134,EV 플랫폼 인프라 보안 점검 요청의 건,보안점검,O,O,O
2,22년,2월,Global,1센터,ALL,완료,KRCLOUD-4377,글로벌 CCS OS 보안 설정 관련 확인 요청의 건,보안점검,O,O,O
3,22년,2월,CN,1센터,STG,완료,CNCLOUD-1801,공신부 점검 지원 요청 드립니다.,보안점검,O,O,O
4,22년,2월,CN,1센터,PRD,완료,CNCLOUD-1801,공신부 점검 지원 요청 드립니다.,보안점검,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...,...
77,22년,7월,KR,1센터,PRD,완료,KRCLOUD-5299,MongoDB 보안점검 요청,보안점검,O,O,O
78,22년,7월,KR,1센터,STG,진행중,KRCLOUD-5205,"LSC, CPW DB 서버에 대한 보안점검 (OS , DB) 부탁드립니다. OPEN",보안점검,O,O,O
79,22년,7월,KR,1센터,PRD,진행중,KRCLOUD-5205,"LSC, CPW DB 서버에 대한 보안점검 (OS , DB)",보안점검,O,O,O
80,22년,7월,EU,1센터,PRD,진행중,EUCLOUD-2723,2022년 유럽 CCS 인프라(VM) 리스트 및 VM별 적용 보안솔루션 현황,보안점검,O,O,O


In [60]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart1(source)
    chart.save(f'./charts/sm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/sm_{region}_{y}_{m}_{d}_2.png')

In [61]:
# Document
document.add_paragraph('보안 관리', style='List Number')
document.add_paragraph(f'총 {completed+incompleted} (완료: {completed} / 미완료: {incompleted})', style='List Bullet')

if region == 'All':
    document.add_picture(f'./charts/sm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Document
addTable3(df_filtered, regex, month)

p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/sm_{region}_{y}_{m}_{d}_2.png')
document.add_page_break()

In [62]:
document.save('demo.docx')

## 1.10 하드웨어 정기점검

In [63]:
base_path = r"./data/1_10_202207_정기점검_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [64]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart1(source)
    chart.save(f'./charts/rc_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/rc_{region}_{y}_{m}_{d}_2.png')

In [65]:
# Document
document.add_paragraph('정기점검', style='List Number')
document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed})', style='List Bullet')

if region == 'All':
    document.add_picture(f'./charts/rc_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Document
document.add_paragraph('상세 현황', style='List Bullet')
addTable3(df_filtered, regex, month)

p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/rc_{region}_{y}_{m}_{d}_2.png')   
document.add_page_break()

## 1.11 데이터베이스_라이선스_관리

In [66]:
base_path = r"./data/1_11_202207_데이터베이스_라이선스_관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O') ]
#temp_license = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O') & (df['라이선스']== '임시')]
# Pivoting
pivot, total = getPivotTable(df, month)

df_ = df_filtered[ (df_filtered['월']==month) ]
#total = len(df_)

df_origin = df_filtered[(df_filtered['월']==month) & (df_filtered['라이선스']== '정식')]
origin = len(df_origin)

df_temp = df_filtered[(df_filtered['월']==month)& (df_filtered['라이선스']== '임시')]
temp = len(df_temp)
# print(total)
# df

In [67]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        chart = getStackedHBarChart1(source)
    chart.save(f'./charts/db_license_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# source = df_filtered
source = df_
chart = getBarChart4s(source, month)
chart.save(f'./charts/db_license_{region}_{y}_{m}_{d}_2.png')


# Chart - 라인 차트
source2 = df_filtered
chart = getLineChart4s(source2, month)
chart.save(f'./charts/db_license_{region}_{y}_{m}_{d}_3.png')

WARN Invalid field type "nominal" for aggregate: "count", using "quantitative" instead.
WARN Invalid field type "nominal" for aggregate: "count", using "quantitative" instead.
WARN Invalid field type "nominal" for aggregate: "count", using "quantitative" instead.
WARN Invalid field type "nominal" for aggregate: "count", using "quantitative" instead.
WARN text dropped as it is incompatible with "point".
WARN Invalid field type "nominal" for aggregate: "count", using "quantitative" instead.
WARN Invalid field type "nominal" for aggregate: "count", using "quantitative" instead.


In [68]:
# Document
document.add_paragraph('데이터베이스 라이선스 관리', style='List Number')
#document.add_paragraph(f'총 {total}건', style='List Bullet')
document.add_paragraph(f'총 {origin+temp}건 (정식 라이선스: {origin}, 임시 라이선스: {temp})', style='List Bullet')


if region == 'All':
    document.add_picture(f'./charts/db_license_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Document
document.add_paragraph('임시 라이선스 사용 내역', style='List Bullet')
addTable_DB_license(df_temp, regex, month)

# addTable(df_filtered, regex, month)
document.add_paragraph('유형별 라이선스 수', style='List Bullet')
document.add_picture(f'./charts/db_license_{region}_{y}_{m}_{d}_2.png')
document.add_paragraph('월별 추세', style='List Bullet')
document.add_picture(f'./charts/db_license_{region}_{y}_{m}_{d}_3.png')
document.add_page_break()

In [69]:
document.save('demo.docx')

## Save Document

In [70]:
document.save(f'월간보고서_{region}_contents_{y}_{m}_{d}.docx')

## 보고서 표지 (커버 쉬트)

In [71]:
# Cover Page
from docxtpl import DocxTemplate

# DRM 방지(cover-page-template.docx)
file_path=os.path.abspath("")
srcpath = findfile("cover-page-template.docx.org", file_path)
dir, file = os.path.split(srcpath)
shutil.copy2(srcpath, dir+"\cover-page-template.docx")

doc = DocxTemplate("cover-page-template.docx")
month_split = month.strip("월")

context = { 
    'Service' : "HMC CCS Infra 운영/관리",      
    'Title' : "Monthly Report",    
    'Region' : region,        
    'Year' : "2022",
    'Month' : f"{month_split}",
    'Department' : "클라우드운영센터"
}
doc.render(context)

doc.save(f'월간보고서_cover_{region}.docx')

In [72]:
from docx import Document
from docxcompose.composer import Composer

composer = None
doc1 = Document(f'월간보고서_cover_{region}.docx')
composer = Composer(doc1)

doc2 = Document(f'월간보고서_{region}_contents_{y}_{m}_{d}.docx')
composer.append(doc2)

composer.save(f'월간보고서_{region}_{y}_{m}_{d}.docx')
